In [35]:
import pandas as pd
import numpy as np

import geopandas as gpd
import geopy

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [36]:
locator = Nominatim(user_agent='javier', timeout=20)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.001)

def location_info(x):
    data = locator.geocode(x).raw
    data_converted = pd.json_normalize(data).squeeze()
    return data_converted

In [40]:
df = pd.read_excel('output/distribucion_sociodemografica/distribucion_sociodemografica.xlsx')

In [41]:
df['region_pais'] = 'Region ' + df['GLOSA_REGION_RESIDENCIA'] + ', Chile'
df['comuna_region_pais'] = df['GLOSA_COMUNA_RESIDENCIA'] + ', ' + df['region_pais']

In [ ]:
regiones_unicas = pd.Series(df['region_pais'].unique()).apply(location_info)